

## 🔁 What is a PUT Request?

### 🧠 Definition:

A `PUT` request is an HTTP method used to **replace** or **update** a resource at a known location (usually using an ID).

✅ It **replaces the full object** (unlike PATCH, which does partial update).

---

## 🎯 Purpose of PUT

| Purpose                       | Real-Life Example                |
| ----------------------------- | -------------------------------- |
| Replace entire resource       | Update full patient record       |
| Ensure idempotency            | Multiple same PUTs = same result |
| Maintain strict state control | Overwrite data if needed         |

---

## 🔧 Operations You Can Perform

* Replace an entire patient profile
* Enforce re-validation on updated fields
* Trigger recomputation of **computed fields**
* Prevent missing data or defaulting to old values (since PUT expects full payload)

---

## 🏥 Scenario: Patient Info Update via PUT

Let’s assume your system already has this patient:

```json
{
  "id": 101,
  "name": "Mahbub Hossain",
  "age": 28,
  "weight_kg": 72,
  "height_cm": 170
}
```

Now, the user wants to **update the whole record** — they grew taller or lost weight.

---

## ✅ FastAPI Code Example – `PUT /patients/{id}`

### ✅ 1. Pydantic Model

```python
from fastapi import FastAPI, Path
from pydantic import BaseModel, Field, computed_field

app = FastAPI()

class Patient(BaseModel):
    name: str = Field(..., min_length=2)
    age: int = Field(..., ge=0, le=120)
    weight_kg: float = Field(..., gt=0)
    height_cm: float = Field(..., gt=0)

    @computed_field
    @property
    def bmi(self) -> float:
        height_m = self.height_cm / 100
        return round(self.weight_kg / (height_m ** 2), 2)
```

---

### ✅ 2. PUT Endpoint

```python
# Simulated DB
mock_db = {
    101: {
        "name": "Mahbub Hossain",
        "age": 28,
        "weight_kg": 72,
        "height_cm": 170
    }
}

@app.put("/patients/{patient_id}")
def update_patient(patient_id: int = Path(..., gt=0), updated_data: Patient = ...):
    if patient_id not in mock_db:
        return {"error": f"Patient with ID {patient_id} not found"}

    # Step 1: Replace data in DB
    mock_db[patient_id] = updated_data.model_dump()

    # Step 2: Return updated data including computed BMI
    return {
        "message": f"Patient {patient_id} updated successfully!",
        "bmi": updated_data.bmi,
        "data": updated_data.model_dump()
    }
```

---

## ✅ Sample PUT Request:

**Endpoint**: `PUT /patients/101`

**Request Body:**

```json
{
  "name": "Mahbub Hossain",
  "age": 29,
  "weight_kg": 75,
  "height_cm": 172
}
```

### ✅ Response:

```json
{
  "message": "Patient 101 updated successfully!",
  "bmi": 25.35,
  "data": {
    "name": "Mahbub Hossain",
    "age": 29,
    "weight_kg": 75.0,
    "height_cm": 172.0
  }
}
```

👀 Notice: BMI is **automatically recomputed** because we updated weight/height.

---

## 🧪 Another Use Case: Recompute on Change

Let’s say you have another **computed field**: health category based on BMI.

### ✅ Add to Model:

```python
    @computed_field
    @property
    def health_category(self) -> str:
        bmi = self.bmi
        if bmi < 18.5:
            return "Underweight"
        elif bmi < 25:
            return "Normal"
        elif bmi < 30:
            return "Overweight"
        else:
            return "Obese"
```

Now your PUT response will include:

```json
{
  "bmi": 25.35,
  "health_category": "Overweight"
}
```

---

# 🧠 Why PUT instead of PATCH?

| PUT                    | PATCH                         |
| ---------------------- | ----------------------------- |
| Replaces full resource | Partially updates resource    |
| All fields expected    | Only changed fields needed    |
| Idempotent             | May not be idempotent         |
| Validates full model   | Validates only updated subset |

You use PUT when:

* You want full control over data consistency
* Your UI sends the full form
* You want to revalidate **all fields**

---